In [1]:
import pandas as pd
import librosa
import numpy as np
import os
import cv2


In [2]:
metadata_path = '/home/karan/JupyterKaran/SoundClassifier/UrbanSound8K/metadata//UrbanSound8K.csv'  # Update this path
audio_path = '/home/karan/JupyterKaran/SoundClassifier/UrbanSound8K/audio'   # Update this path


In [4]:
def extract_spectrogram_features(metadata_path, audio_path, img_size=(64, 64)):
    metadata = pd.read_csv(metadata_path)
    emergency_classes = ['car_horn', 'siren']
    metadata['emergency'] = metadata['class'].apply(lambda x: 1 if x in emergency_classes else 0)
    
    features = []
    labels = []
    
    for index, row in metadata.iterrows():
        file_path = os.path.join(audio_path, f"fold{row['fold']}", row['slice_file_name'])
        
        try:
            audio, sample_rate = librosa.load(file_path, sr=None)
            
            # Pad/crop audio to ensure consistent duration
            target_samples = int(1.0 * sample_rate)  # 1-second audio
            if len(audio) < target_samples:
                audio = np.pad(audio, (0, target_samples - len(audio)), mode='constant')
            else:
                audio = audio[:target_samples]
            
            # Generate spectrogram
            spect = librosa.feature.melspectrogram(
                y=audio,
                sr=sample_rate,
                n_fft=2048,
                hop_length=512,
                n_mels=img_size[0]
            )
            spect = librosa.power_to_db(spect, ref=np.max)
            
            # Resize to exact dimensions (64, 64)
            spect = cv2.resize(spect, (img_size[1], img_size[0]))
            
            # Normalize to [0, 1]
            spect = (spect - spect.min()) / (spect.max() - spect.min())
            
            features.append(spect)
            labels.append(row['emergency'])
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            continue
    
    # Convert to numpy arrays with explicit shape
    features = np.array(features).reshape(-1, img_size[0], img_size[1], 1)
    labels = np.array(labels)
    
    return features, labels

In [5]:
X, y = extract_spectrogram_features(metadata_path, audio_path, img_size=(64, 64))


In [6]:
X = X.reshape(-1, 64, 64, 1)  

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    # Convolutional layers
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Fully connected layers
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

2025-04-04 16:22:41.793066: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-04 16:22:41.793641: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-04 16:22:41.795899: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-04 16:22:41.800928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743763961.809870  120732 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743763961.81

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 682,753 (2.60 MB)

 Trainable params: 682,753 (2.60 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test)
)

model.save('emergency_sound_model.h5')

Epoch 1/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.9921 - loss: 0.0251 - val_accuracy: 0.9725 - val_loss: 0.2219
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.9987 - loss: 0.0046 - val_accuracy: 0.9697 - val_loss: 0.2419
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.9972 - loss: 0.0081 - val_accuracy: 0.9651 - val_loss: 0.1967
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.9893 - loss: 0.0304 - val_accuracy: 0.9748 - val_loss: 0.2184
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.9994 - loss: 0.0020 - val_accuracy: 0.9685 - val_loss: 0.2298
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.9972 - loss: 0.0082 - val_accuracy: 0.9668 - val_loss: 0.2950
Epoch 7/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.9982 - loss: 0.0108 - val_accuracy: 0.9725 - val_loss: 0.2161
Epoch 8/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.9988 - loss: 0.0057 - val_accu

In [39]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

# Predict on new audio
def predict_audio(file_path, model, img_size=(64, 64)):
    audio, sr = librosa.load(file_path, sr=None)
    
    # Generate spectrogram
    spect = librosa.feature.melspectrogram(y=audio, sr=sr)
    spect = librosa.power_to_db(spect, ref=np.max)
    spect = cv2.resize(spect, img_size)
    spect = (spect - spect.min()) / (spect.max() - spect.min())
    
    # Reshape for model
    spect = spect.reshape(1, 64, 64, 1)
    
    # Predict
    prob = model.predict(spect)[0][0]
    return "Emergency Sound" if prob > 0.5 else "Normal Sound"

55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9645 - loss: 0.3262
Test Accuracy: 0.97


In [40]:
import pyaudio

# Constants
RATE = 22050
CHUNK = 1024  # Standard chunk size
BUFFER_SECONDS = 1.0

# Calculate optimal buffer size
CHUNKS_PER_BUFFER = int(np.ceil(RATE * BUFFER_SECONDS / CHUNK))
BUFFER_SIZE = CHUNKS_PER_BUFFER * CHUNK  # Now perfectly divisible

print(f"Adjusted buffer: {BUFFER_SIZE} samples ({BUFFER_SIZE/RATE:.3f} sec)")
print(f"Chunks per buffer: {CHUNKS_PER_BUFFER}")


Adjusted buffer: 22528 samples (1.022 sec)
Chunks per buffer: 22


In [41]:
audio_buffer = np.zeros(BUFFER_SIZE, dtype=np.float32)
ptr = 0

In [42]:
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paFloat32,
                channels=1,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

audio_buffer = np.zeros(BUFFER_SIZE, dtype=np.float32)
write_ptr = 0


In [20]:
print("🎙️ Listening for emergency sounds... (Press Ctrl+C to stop)")

🎙️ Listening for emergency sounds... (Press Ctrl+C to stop)


In [43]:
try:
    while True:
        # Read chunk
        data = stream.read(CHUNK, exception_on_overflow=False)
        chunk = np.frombuffer(data, dtype=np.float32)
        
        # Write to buffer
        audio_buffer[write_ptr:write_ptr+CHUNK] = chunk
        write_ptr += CHUNK
        
        # Process when buffer is full
        if write_ptr >= BUFFER_SIZE:
            print("\nBuffer full - ready for processing!")
            print(f"Max amplitude: {np.max(audio_buffer):.3f}")
            
            # Reset for next cycle
            write_ptr = 0
            audio_buffer.fill(0)

except KeyboardInterrupt:
    stream.stop_stream()
    stream.close()
    p.terminate()



Buffer full - ready for processing!
Max amplitude: 0.412

Buffer full - ready for processing!
Max amplitude: 0.429

Buffer full - ready for processing!
Max amplitude: 1.159

Buffer full - ready for processing!
Max amplitude: 1.156

Buffer full - ready for processing!
Max amplitude: 1.195

Buffer full - ready for processing!
Max amplitude: 1.195

Buffer full - ready for processing!
Max amplitude: 1.031

Buffer full - ready for processing!
Max amplitude: 0.373

Buffer full - ready for processing!
Max amplitude: 0.362

Buffer full - ready for processing!
Max amplitude: 1.069

Buffer full - ready for processing!
Max amplitude: 1.010

Buffer full - ready for processing!
Max amplitude: 0.369

Buffer full - ready for processing!
Max amplitude: 1.009

Buffer full - ready for processing!
Max amplitude: 1.006

Buffer full - ready for processing!
Max amplitude: 0.766

Buffer full - ready for processing!
Max amplitude: 1.010

Buffer full - ready for processing!
Max amplitude: 0.368

Buffer full -

In [22]:
print(model.input_shape)  # Should be (None, 64, 64, 1)

(None, 64, 64, 1)


In [26]:
import pyaudio
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    dev = p.get_device_info_by_index(i)
    print(f"{i}: {dev['name']} (Inputs: {dev['maxInputChannels']})")
p.terminate()

0: sof-hda-dsp: - (hw:0,0) (Inputs: 2)
1: sof-hda-dsp: - (hw:0,3) (Inputs: 0)
2: sof-hda-dsp: - (hw:0,4) (Inputs: 0)
3: sof-hda-dsp: - (hw:0,5) (Inputs: 0)
4: sof-hda-dsp: - (hw:0,6) (Inputs: 2)
5: sof-hda-dsp: - (hw:0,7) (Inputs: 2)
6: sof-hda-dsp: - (hw:0,31) (Inputs: 0)
7: sysdefault (Inputs: 128)
8: pipewire (Inputs: 64)
9: dmix (Inputs: 0)
10: default (Inputs: 64)


ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.front
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround40
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround41
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround50
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround51
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround71
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2721:(snd

In [29]:
import pyaudio
import numpy as np

# Audio settings
CHUNK = 1024
FORMAT = pyaudio.paFloat32
CHANNELS = 1
RATE = 44100  # Try higher rate if 22050 fails
DEVICE_INDEX = None  # Set to your mic index if needed

p = pyaudio.PyAudio()

# Print available devices again
print("\nAvailable devices:")
for i in range(p.get_device_count()):
    dev = p.get_device_info_by_index(i)
    print(f"{i}: {dev['name']}")

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                input_device_index=DEVICE_INDEX,
                frames_per_buffer=CHUNK)

print("\nSpeak into your microphone...")

try:
    while True:
        data = stream.read(CHUNK, exception_on_overflow=False)
        audio = np.frombuffer(data, dtype=np.float32)
        volume = np.abs(audio).mean()
        print(f"Input level: {volume:.4f}", end='\r')
        if volume > 0.01:  # If you see this, audio is working
            print("\nAudio detected!")
            break
except KeyboardInterrupt:
    pass
finally:
    stream.stop_stream()
    stream.close()
    p.terminate()


Available devices:
0: sof-hda-dsp: - (hw:0,0)
1: sof-hda-dsp: - (hw:0,3)
2: sof-hda-dsp: - (hw:0,4)
3: sof-hda-dsp: - (hw:0,5)
4: sof-hda-dsp: - (hw:0,7)
5: sof-hda-dsp: - (hw:0,31)
6: sysdefault
7: pipewire
8: dmix
9: default

Speak into your microphone...
Input level: 0.0914
Audio detected!


ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.front
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround40
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround41
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround50
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround51
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround71
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2721:(snd

In [37]:
import pyaudio
import numpy as np

# Audio settings
CHUNK = 1024
FORMAT = pyaudio.paFloat32
CHANNELS = 1
RATE = 44100  # Try higher rate if 22050 fails
DEVICE_INDEX = None  # Set to your mic index if needed

p = pyaudio.PyAudio()

# Print available devices again
print("\nAvailable devices:")
for i in range(p.get_device_count()):
    dev = p.get_device_info_by_index(i)
    print(f"{i}: {dev['name']}")

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                input_device_index=DEVICE_INDEX,
                frames_per_buffer=CHUNK)

print("\nSpeak into your microphone...")

try:
    while True:
        data = stream.read(CHUNK, exception_on_overflow=False)
        audio = np.frombuffer(data, dtype=np.float32)
        volume = np.abs(audio).mean()
        print(f"Input level: {volume:.4f}", end='\r')
        if volume > 0.01:  # If you see this, audio is working
            print("\nAudio detected!")
            break
except KeyboardInterrupt:
    pass
finally:
    stream.stop_stream()
    stream.close()
    p.terminate()


Available devices:
0: sof-hda-dsp: - (hw:0,0)
1: sof-hda-dsp: - (hw:0,3)
2: sof-hda-dsp: - (hw:0,4)
3: sof-hda-dsp: - (hw:0,5)
4: sof-hda-dsp: - (hw:0,7)
5: sof-hda-dsp: - (hw:0,31)
6: sysdefault
7: pipewire
8: dmix
9: default

Speak into your microphone...
Input level: 0.2977
Audio detected!


In [44]:
# Circular buffer implementation
audio_buffer = np.zeros(BUFFER_SIZE, dtype=np.float32)
write_ptr = 0
chunk_counter = 0

def process_audio(buffer):
    # Create spectrogram
    spect = librosa.feature.melspectrogram(y=buffer, sr=RATE, n_mels=64)
    spect = librosa.power_to_db(spect, ref=np.max)
    spect = cv2.resize(spect, (64, 64))
    
    # Safe normalization
    spect = np.nan_to_num(spect)
    if np.ptp(spect) > 0:
        spect = (spect - spect.min()) / np.ptp(spect)
    else:
        spect = np.zeros((64, 64))
    return np.expand_dims(spect, axis=-1)


In [45]:

# Constants
RATE = 22050
CHUNK = 1024  # Must be power of 2 (512, 1024, 2048)
BUFFER_SECONDS = 1.0
BUFFER_SIZE = int(RATE * BUFFER_SECONDS)
THRESHOLD = 0.7

In [46]:
try:
    while True:
        try:
            # Read audio chunk
            data = stream.read(CHUNK, exception_on_overflow=False)
            chunk = np.frombuffer(data, dtype=np.float32)
            
            # Write to circular buffer
            if write_ptr + CHUNK <= BUFFER_SIZE:
                audio_buffer[write_ptr:write_ptr+CHUNK] = chunk
            else:
                remaining = BUFFER_SIZE - write_ptr
                audio_buffer[write_ptr:] = chunk[:remaining]
                audio_buffer[:CHUNK-remaining] = chunk[remaining:]
            
            write_ptr = (write_ptr + CHUNK) % BUFFER_SIZE
            chunk_counter += 1
            
            print(f"{RATE} samples/sec ÷ {CHUNK} samples/chunk = {RATE/CHUNK:.2f} chunks/sec")
            print(f"Exact buffer: {BUFFER_SIZE} samples = {BUFFER_SIZE/CHUNK} chunks")
            # Process every full buffer cycle
            if chunk_counter >= BUFFER_SIZE // CHUNK:
                spect = process_audio(audio_buffer)
                prob = model.predict(spect[np.newaxis, ...], verbose=0)[0][0]
                
                if prob > THRESHOLD:
                    print(f"🚨 EMERGENCY ({prob:.0%} confidence)")
                else:
                    print(f"Normal ({prob:.0%})", end='\r')
                
                chunk_counter = 0
                
        except Exception as e:
            print(f"\nError: {str(e)}")
            time.sleep(0.1)
            # Reset buffer on serious errors
            audio_buffer.fill(0)
            write_ptr = 0
            chunk_counter = 0

except KeyboardInterrupt:
    print("\nStopping...")
    stream.stop_stream()
    stream.close()
    p.terminate()


Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream closed

Error: [Errno -9988] Stream

In [3]:
import tensorflow as tf

# Load your trained model
model = tf.keras.models.load_model('emergency_sound_model.h5')

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Apply optimizations (optional but recommended)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# For further size reduction and speed (optional)
# converter.target_spec.supported_types = [tf.float16]

# Convert the model
tflite_model = converter.convert()

# Save the TFLite model
with open('emergency_sound_classifier.tflite', 'wb') as f:
    f.write(tflite_model)

2025-04-11 19:43:42.864445: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-11 19:43:42.865006: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-11 19:43:42.868424: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-11 19:43:42.875237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744380822.887226   47566 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744380822.89

INFO:tensorflow:Assets written to: /tmp/tmp37usm71d/assets


INFO:tensorflow:Assets written to: /tmp/tmp37usm71d/assets


Saved artifact at '/tmp/tmp37usm71d'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139037023710352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139037023711120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139037023713040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139037023710736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139037023714384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139037023715152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139037023714768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139036951216976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139036951218128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139036951219280: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1744380824.686525   47566 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1744380824.686541   47566 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-04-11 19:43:44.686906: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp37usm71d
2025-04-11 19:43:44.687281: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-04-11 19:43:44.687287: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp37usm71d
I0000 00:00:1744380824.691403   47566 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-04-11 19:43:44.692189: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-04-11 19:43:44.724181: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp37usm71d
2025-04-11 19:43:44.735274: I tensorflow/cc/saved_model/loader.cc:471] SavedModel 